# 少量示例的提示词模板

FewShotPromptTemplate:与PromptTemplate一起使用

FewShotChatMessagePromptTemplate:与ChatPromptTemplate一起使用

Example selectors(示例选择器)：

## 1、FewShotPromptTemplate的使用

示例1：未提供示例的情况

In [1]:
from sys import prefix

import dotenv
from langchain.agents.mrkl.prompt import SUFFIX
from langchain.chains.constitutional_ai.prompts import examples
from langchain_core.prompts import FewShotPromptTemplate
from langchain_openai import ChatOpenAI
import os

from openai.resources.containers.files import content
from requests import Response

dotenv.load_dotenv()

# 初始化聊天模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    streaming=True,
)

res = chat_model.invoke("2 🐦 9是多少？")
print(res.content)

“2 🐦 9是多少？”这个问题中的“🐦”是一个表情符号（小鸟），在数学表达式中并没有标准的数学含义。因此，这个式子本身不是一个常规的数学运算。

不过，这类问题有时是网络上的谜语、谐音梗或脑筋急转弯。我们可以尝试从其他角度理解：

1. **谐音角度**：  
   - “🐦”读作“鸟”，在中文里有时被用作“叼”或某些方言中的替代字，但和数字结合不太常见。
   - “2鸟9”听起来不像常见的谐音短语。

2. **表情符号代指**：  
   - 有时“🐦”在网络语言中可能代表“推特”（Twitter 的 logo 是小鸟），但“2 推特 9”也没有明确意义。

3. **可能是输入错误或替代符号**：  
   - 也许“🐦”原本想输入的是某个运算符，比如“+”、“-”、“×”等，但误用了表情符号。
     - 如果是“2 + 9” = 11  
     - “2 × 9” = 18  
     - “2^9” = 512（如果是指数）

4. **趣味解读**：  
   - 有人开玩笑说“两只鸟加九只鸟等于十一只鸟”，那答案就是 11（把 🐦 当作单位）。
   - 所以“2 🐦 9”可能意为“2 只鸟和 9”，但逻辑不通；更可能是“2 🐦 + 9 🐦 = 11 🐦”。

**最合理的猜测**：这是一个用表情符号代替文字的趣味题，可能原意是“2 + 9 = ?”，而“🐦”只是装饰或误用。  
因此，答案很可能是：**11**。

如果你有更多上下文（比如这是来自某个游戏、谜题或特定语境），欢迎补充！


举例2：使用FewShotPromptTemplate

In [3]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

example_prompt = PromptTemplate.from_template(
    template="input: {input}\noutput: {output}",
)

examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京今天下雨吗", "output": "南京市"},
    {"input": "上海现在气温多少度", "output": "上海市"},
]

few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="input: {input}\noutput:", # 声明在示例后面的提示词模板
    input_variables=["input"],
)

few_shot_template.invoke({
    "input": "广州今天的天气如何？"
})

StringPromptValue(text='input: 北京天气怎么样\noutput: 北京市\n\ninput: 南京今天下雨吗\noutput: 南京市\n\ninput: 上海现在气温多少度\noutput: 上海市\n\ninput: 广州今天的天气如何？\noutput:')

调用大模型以后：

In [6]:
import dotenv
from langchain_openai import ChatOpenAI
import os

dotenv.load_dotenv()

# 初始化聊天模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    streaming=True,
)



In [7]:
chat_model.invoke(few_shot_template.invoke({
    "input": "广州今天的天气如何？"
}))

AIMessage(content='广州市', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen3-max'}, id='run--fa39b79a-b099-4d21-94d7-a43b71392e5d-0')

## 2、FewShotChatMessagePromptTemplate的使用

举例1：实例化

In [2]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"input": "1+1等于几？", "output": "1+1等于2"},
    {"input": "法国的首都是？", "output": "巴黎"},
]

msg_examples_prompt = ChatPromptTemplate.from_messages([
    ("user", "问题： {input}"),
    ("assistant", "回答： {output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_examples_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 问题： 1+1等于几？
AI: 回答： 1+1等于2
Human: 问题： 法国的首都是？
AI: 回答： 巴黎


举例2：结合大模型使用

In [3]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"input": "2 🐦 2", "output": "4"},
    {"input": "2 🐦 3", "output": "8"},
]

examples_prompt = ChatPromptTemplate.from_messages([
    ("user", "计算： {input}"),
    ("assistant", "结果： {output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=examples_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个数学奇才。"),
    few_shot_prompt,
    ("user", "计算： {input}"),
])

import dotenv
from langchain_openai import ChatOpenAI
import os

dotenv.load_dotenv()

# 初始化聊天模型
chat_model = ChatOpenAI(
    model="qwen3-max",
    temperature=0,
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    streaming=True,
)

chat_model.invoke(final_prompt.invoke(input="3 🐦 4")).content

'结果： 81'

## 3、示例选择器(Example Selectors)

示例1：

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# 聊天模型
from langchain_community.chat_models import ChatTongyi
# Embedding 模型
from langchain_community.embeddings import DashScopeEmbeddings
import os
import dotenv

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    temperature=0.7,
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY"),
)

embeddings = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY")
)

examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：穆罕默德·阿里去世时多大年纪？
        中间答案：穆罕默德·阿里去世时享年74岁。
        """,
    },
    {
        "question": "爱因斯坦和牛顿谁更早提出相对论？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：牛顿提出的力学理论是什么时候的事？
        中间答案：牛顿在17世纪提出了经典力学理论。
        """,
    },
    {
        "question": "莎士比亚和狄更斯谁更早出生？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：莎士比亚出生于哪一年？
        中间答案：莎士比亚出生于1564年。
        """,
    },
    {
        "question": "乔布斯和盖茨谁更早创办公司？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：比尔·盖茨创办微软公司是哪一年？
        中间答案：比尔·盖茨在1975年创办了微软公司。
        """,
    },
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1,
)

question = "尼古拉·特斯拉和托马斯·爱迪生谁更早发明交流电？"
selected_examples = example_selector.select_examples(
    {"question": question}
)
print(f"与问题'{question}'最相关的示例是：{selected_examples}")


与问题'尼古拉·特斯拉和托马斯·爱迪生谁更早发明交流电？'最相关的示例是：[{'answer': '\n        接下来还需要问什么问题吗？\n        追问：牛顿提出的力学理论是什么时候的事？\n        中间答案：牛顿在17世纪提出了经典力学理论。\n        ', 'question': '爱因斯坦和牛顿谁更早提出相对论？'}]


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
# 聊天模型
from langchain_community.chat_models import ChatTongyi
# Embedding 模型
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

embeddings = DashScopeEmbeddings(
    model="text-embedding-v4",
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY"),
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2,
)

similar_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    prefix="请给出以下词语的反义词。",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"],
)

response = similar_prompt.invoke({"word": "忧郁"})
print(response.text)

请给出以下词语的反义词。

Input: 高兴
Output: 悲伤

Input: 阳光
Output: 阴暗

Input: 忧郁
Output:


## 4、管道提示词模板(PipelinePromptTemplate)

In [6]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

# 阶段1：分析问题，提取关键要素
analysis_template = PromptTemplate.from_template("""
分析这个问题：{question}
关键要素：
""")

# 阶段2：基于关键要素搜索资料
retrieval_template = PromptTemplate.from_template("""
基于以下要素搜索资料：
{analysis_result}
搜索关键词：
""")

# 阶段3：综合资料，给出最终答案
answer_template = PromptTemplate.from_template("""
综合以下信息回答问题：
{retrieval_result}
最终答案：
""")

# 构建管道
pipeline = PipelinePromptTemplate(
    final_prompt=answer_template,
    pipeline_prompts=[
        ("analysis_result", analysis_template),
        ("retrieval_result", retrieval_template),
    ]
)

print(pipeline.format(question="量子计算的优势是什么？"))


综合以下信息回答问题：

基于以下要素搜索资料：

分析这个问题：量子计算的优势是什么？
关键要素：

搜索关键词：

最终答案：



C:\Users\yzy\AppData\Local\Temp\ipykernel_144940\2008939700.py:25: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline = PipelinePromptTemplate(


In [8]:
from langchain_core.prompts.prompt import PromptTemplate

# 阶段1：分析问题，提取关键要素
analysis_template = PromptTemplate.from_template("""
分析这个问题：{question}
关键要素：
""")

# 阶段2：基于关键要素搜索资料
retrieval_template = PromptTemplate.from_template("""
基于以下要素搜索资料：
{analysis_result}
搜索关键词：
""")

# 阶段3：综合资料，给出最终答案
answer_template = PromptTemplate.from_template("""
综合以下信息回答问题：
{retrieval_result}
最终答案：
""")

# 逐步执行管道提示
pipeline_prompts = [
    ("analysis_result", analysis_template),
    ("retrieval_result", retrieval_template),
]

my_input = {"question": "量子计算的优势是什么？"}

for name, prompt in pipeline_prompts:
    result = prompt.invoke(my_input).to_string()
    my_input[name] = result

my_output = answer_template.invoke(my_input).to_string()
print(my_output)


综合以下信息回答问题：

基于以下要素搜索资料：

分析这个问题：量子计算的优势是什么？
关键要素：

搜索关键词：

最终答案：

